In [1]:
SMAUG128 = {}
SMAUG128['n'] = 256
SMAUG128['p'] = 256
SMAUG128['q'] = 1024
SMAUG128['p_prime'] = 32
SMAUG128['k'] = 2
SMAUG128['h_s'] = 140
SMAUG128['h_r'] = 132
SMAUG128['sigma'] = 1.0625

SMAUG192 = {}
SMAUG192['n'] = 256
SMAUG192['p'] = 256
SMAUG192['q'] = 2048
SMAUG192['p_prime'] = 256
SMAUG192['k'] = 3
SMAUG192['h_s'] = 198
SMAUG192['h_r'] = 151
SMAUG192['sigma'] = 1.453713

SMAUG256 = {}
SMAUG256['n'] = 256
SMAUG256['p'] = 256
SMAUG256['q'] = 2048
SMAUG256['p_prime'] = 64
SMAUG256['k'] = 5
SMAUG256['h_s'] = 176
SMAUG256['h_r'] = 160
SMAUG256['sigma'] = 1.0625

#DGdict = {0: 403163305, 1: 258898250, 2: 68560420, 3: 7487107, 4: 337172, 5: 6262, 6: 48} # sigma = 1.0625
#DGdict = {0: 8993, 1: 7098, 2: 3490, 3: 1069, 4: 204, 5: 24, 6: 2} # sigma = 1.453713

# TiGER v2.1
TiGER256 = {}
TiGER256['n'] = 1024
TiGER256['q'] = 256
TiGER256['p'] = 128
TiGER256['k1'] = 128
TiGER256['k2'] = 4
TiGER256['h_s'] = 196
TiGER256['h_r'] = 196
TiGER256['h_e'] = 32
TiGER256['f'] = 5
TiGER256['thres'] = 256/2

TiGER192 = {}
TiGER192['n'] = 1024
TiGER192['q'] = 256
TiGER192['p'] = 128
TiGER192['k1'] = 64
TiGER192['k2'] = 4
TiGER192['h_s'] = 132
TiGER192['h_r'] = 132
TiGER192['h_e'] = 32
TiGER192['f'] = 5
TiGER192['thres'] = 256/2

TiGER128 = {}
TiGER128['n'] = 512
TiGER128['q'] = 256
TiGER128['p'] = 128
TiGER128['k1'] = 64 
TiGER128['k2'] = 16
TiGER128['h_s'] = 142
TiGER128['h_r'] = 110
TiGER128['h_e'] = 32
TiGER128['f'] = 3
TiGER128['thres'] = 256/2

In [2]:
RR = RealField(5000)
R.<x> = LaurentPolynomialRing(QQ)
sp_t_poly = 1/2*x**-1 + 1/2*x

In [3]:
def unif_error_poly_(p, q):
    
    err_max = int(q/p/2)
    err_list = [i for i in range(err_max+1)]
    poly = 0
    for i in err_list:
        if i == 0 :
            poly += 1
        elif i == err_max :
            poly += 1/2*x**err_max + 1/2*x**(-err_max)
        else :
            poly += x**i + x**(-i)
    
    return poly/(2*err_max)


def e2_poly_(p_prime, q): 
    
    err_nums = q/p_prime
    zero_pos = int(err_nums/2-1)
    poly = 0
    for i in range(err_nums):
        if not i == zero_pos:
            poly += x**(i-zero_pos)
        else :
            poly += 1
    
    return poly/err_nums
    
def dist_mult_(poly1, poly2):
    
    poly = 0
    expo_u = vector(poly1.exponents())
    expo_v = vector(poly2.exponents())
    coef_u = vector(poly1.coefficients())
    coef_v = vector(poly2.coefficients())
    
    for i, rows in enumerate(expo_u.tensor_product(expo_v)):
        for j, elts in enumerate(rows):
            poly += coef_u.tensor_product(coef_v)[i][j]*x**elts
    
    return poly

def DGdict_to_DGpoly_(DGdict):
    DGpoly = 0
    for i in DGdict :
        if i == 0 :
            DGpoly += DGdict[i]*x**i
        else :
            DGpoly += DGdict[i]*x**i + DGdict[i]*x**-i
    DGpoly = DGpoly/DGpoly(1)
    
    return DGpoly

def cumul_binom_(p, n, k):
    res = 0
    for i in range(k+1):
        res += binomial(n, i)*(p**i)*(1-p)**(n-i)
        
    return res

def abs_poly(poly):
    for poly_expo in poly.exponents():
        if poly_expo < 0 :
            poly += poly[poly_expo]*(x**(-poly_expo))-poly[poly_expo]*(x**poly_expo)
    
    return poly

In [4]:
def SMAUG_DFP(scheme):
    n = scheme['n']
    p = scheme['p']
    q = scheme['q']
    k = scheme['k']
    p_prime = scheme['p_prime']
    h_s = scheme['h_s']
    h_r = scheme['h_r']
    sigma = scheme['sigma']
    
    if not scheme == SMAUG192 :
        DGdict = {0: 403163305, 1: 258898250, 2: 68560420, 3: 7487107, 4: 337172, 5: 6262, 6: 48}
    else :
        DGdict = {0: 8993, 1: 7098, 2: 3490, 3: 1069, 4: 204, 5: 24, 6: 2}
    
    e_poly = DGdict_to_DGpoly_(DGdict)
    e1_poly = unif_error_poly_(p, q)
    e2_poly = e2_poly_(p_prime, q)

    first = (dist_mult_(e_poly, sp_t_poly))**h_r
    second = (dist_mult_(e1_poly, sp_t_poly))**h_s
    last = e2_poly
    
    bit_err_poly = first*second*last
    
    bit_err_prob = 0
    
    for expo in bit_err_poly.exponents():
        if expo > q/4 :
            bit_err_prob += bit_err_poly[expo]
    
    dfp = RR(1-(1-bit_err_prob)**(n*k))
    
    return dfp.n(100), log(dfp, 2).n(100)


def TiGER_DFP_independency(scheme):
    
    n = scheme['n']
    p = scheme['p']
    q = scheme['q']
    k1 = scheme['k1']
    k2 = scheme['k2']
    h_r = scheme['h_r']
    h_e = scheme['h_e']
    h_s = scheme['h_s']
    thres = scheme['thres']
    f = scheme['f']
    
    uA = unif_error_poly_(p, q)
    
    eB_p = sp_t_poly*(h_e/n)+(1-h_e/n)
    uB_p = unif_error_poly_(k1, q)
    
    
    uB_dp = unif_error_poly_(k2, q)
    eB_dp = sp_t_poly*(h_e/n)+(1-h_e/n)
    
    
    first = (eB_p*uB_p)**h_s
    second = uA**h_r
    last = eB_dp*uB_dp
    
    bit_err_poly1 = (first*second*last)
    bit_err_poly2 = bit_err_poly1
    

    bit_suc_prob = 0
    
    for expo2 in bit_err_poly2.exponents():
        for expo1 in bit_err_poly1.exponents():
            if expo1+expo2 < thres :
                bit_suc_prob += bit_err_poly1[expo1]*bit_err_poly2[expo2]
    
    bit_err_prob = 1-bit_suc_prob
    
    dfp_wo_ecc = RR(1-(1-bit_err_prob)**n)
    dfp_w_ecc = 1-cumul_binom_(bit_err_prob, n, f)
    
    return log(RR(bit_err_prob), 2).n(1000), log(dfp_wo_ecc, 2).n(1000), log(dfp_w_ecc, 2).n(1000)

In [5]:
 TiGER_DFP_independency(TiGER128)

(-23.6025883273439699294321532071270337149061109591619169022152459423421811436086182720622020564107757806603324111480523777104910426914529284209677777936566953659421557199847960134878852632954230649242888671540285888883497603991164135865994886613632719528671387584766319725134734563276401013947670479717,
 -14.6026172659056011302883799960401431492998588342656368499481681572977266949452183065529343251702032018052352049941770773202325465736826774852163060042143277419509005883207262449808897640745349475031296936367870467407854812072408476303418867593896575702850856378555096012867890442597024262526804515035,
 -63.0123070762171271729798238331635015908167833696351450807592064635492601694821360043176945533822343850835593377603625688099724460813577870922822749833259394199331159913216366754166887815082510736861351987076306097109790143926190450946501885531985554588739870638056912662329401174351659075797653922788)

In [6]:
 TiGER_DFP_independency(TiGER192)

(-14.8474319227024736516377714544634813878215287746518362733946789135024474018830071980803511298954686877376085072029580601371310877736181295234857705778492464270016345337769302892309162882486753067703010771239001430082496352194417470696988596212660262470853845783718059248887497331274129034659475288469,
 -4.87239194572015073250306349702181281816375401512295579874528419799474194922196536766262358445497944616750002219172712549809000597348515590516349730942084001209108756246743204030955477090479063740991981890608083506090382197536588620186907119169758968101628150860763818752222727453340229603650561077852,
 -38.6403057268225505615252984392990562388501465130721539061162036217791855433135758256952635087420016199543398249244847419125562431927705796856729510872165274227598805687078307316219961924360393296019516571256150192338100219300633330878022063914585014186765736580030123096645488420070088088181925821866)

In [7]:
 TiGER_DFP_independency(TiGER256)

(-17.4797484602104410707841774840328152889417090377417868025046072546109194832072442013912264482903838122215943929660159335721385756565597967019280221635916825525158349094693225228178623057229800302065877096529569328480441002863709641220313544273987118215774241160122431609081653839869747024883423781959,
 -7.48378388966491278039635051838870470715894707916911657087362929908633252832093429869119178302145182628148584944887348801816115317809018130730978063244627071461212919861482198722282006937018484477045775284867337385979146670366107450674059951310468267870582506906925132914456816654865051656239607644444,
 -54.3984019639688164659046849689478136888059135015163056201646436762221823133804083961044528848835920549152771909987312948193962359873262062918107688285867394588107331065439724674208030229351370073094948066286458742059544392582323435751082061265528853702533237742881185776847792624965258263422919899114)